In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import gspread
import pandas as pd

from google.oauth2 import service_account
from isoweek import Week
import datetime as dt

import pandas_gbq
import requests
import json
import string
from clickhouse_key import ch_key
from clickhouse_driver import Client

In [ ]:
key_path = 'm2-main-cd9ed0b4e222.json'
gbq_credential = service_account.Credentials.from_service_account_file(key_path,)
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
             'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('m2-main-cd9ed0b4e222.json', SCOPES)
gc = gspread.authorize(credentials)

In [ ]:
client = Client(ch_key['db_host'],
               user=ch_key['db_user'],
               password=ch_key['db_password'],
               verify=False,
               database=ch_key['db_name'], settings={'use_numpy': True})

# Трафик по областям (дозапись)

In [ ]:
from datetime import date
today = date.today()
week = today.isocalendar()[1] - 1
week

In [ ]:
re, columns = client.execute(f'''
SELECT
    toISOWeek(date) as week,
    source,
    medium,
    CASE
        WHEN (match (landingpagepath, '(moskva-i-oblast|moskva|moskovskaya-oblast)')) THEN 'msk'
        WHEN (match(landingpagepath, '(sankt-peterburg|leningradskaya-oblast|sankt-peterburg-i-oblast)')) THEN 'spb'
        ELSE 'region'
    END as geo,
    case
        when (medium like '%cpc%' and
              (campaign like '%search_second-building-discovery%' or campaign like '%second-building-smart-kms%')) then 'CPC_UP_Trash'
        WHEN campaign LIKE '%second-building%' then 'CPC_UP'
        WHEN campaign LIKE '%new-build%' then 'CPC_NB'
        else 'Not_cpc'
    end as cmp_class,
    count(distinct dimension1) as uniqs,
    count(distinct dimension4) as sessions
FROM ga.UA_TRAFIC_BIG
where landingpagepath like '%nedvizhimost%' AND toISOYear(date) = 2023 and toISOWeek(date) = {week}
group by week, source, medium, geo, cmp_class
order by week
''', with_column_types = True)

col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)
traf.head(5)

traf = traf[['week', 'geo', 'cmp_class', 'uniqs', 'sessions', 'medium', 'source']]
traf_check = traf.values.tolist()
# traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
# wk.update("A1", traf_check)
wk.append_rows(traf_check, table_range="A2")

In [ ]:
re, columns = client.execute(f'''
with 
a as (
    select
        dateHourMinute,
        dimension4,
        totalEvents
    from ga.RAW_EVENTS
    where match(eventCategory, '^new_.*_sell_.*') AND match(eventlabel, 'UPUNICLOP_.*')
      and toISOYear(date) = 2023 AND toISOWeek(date) = {week}
    group by dimension4, totalEvents, dateHourMinute
),

b as (
    SELECT
        dimension1,
        dimension4
    from ga.UA_TRAFIC_BIG
    where toISOYear(dateHourMinute) = 2023 and toISOWeek(dateHourMinute) = {week}
    group by dimension1, dimension4
)

select
    toISOWeek(dateHourMinute) as week,
    sum(totalEvents) as tot_events,
    count(distinct dimension1) as uns_clops,
    count(distinct dimension4) as ses_clops
from a 
left join b ON a.dimension4 = b.dimension4 
group by week
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)


traf_check = traf.values.tolist()
# traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
# wk.update("I1", traf_check)
wk.append_rows(traf_check, table_range="I2")

In [ ]:
re, columns = client.execute(f'''
with 
pv as(
    SELECT dimension4  
    from ga.PAGE_VIEWS
    where pagepath  like '%nedvizhimost%' AND toISOYear(date) = 2023 AND toISOWeek(date) = {week} 
),

tr as(
    SELECT dateHourMinute, landingpagepath, medium, campaign, source, dimension1, dimension4
    FROM ga.UA_TRAFIC_BIG
    where toISOYear(dateHourMinute) = 2023 AND toISOWeek(dateHourMinute) = {week}
)

SELECT
    toISOWeek(dateHourMinute) as week,
    CASE
        WHEN (match (landingpagepath, '.*nedvizhimost.*(sankt-peterburg|leningradskaya-oblast|sankt-peterburg-i-oblast).*')) THEN 'SPB_UP '
        WHEN (match(landingpagepath, '.*nedvizhimost.*(moskva-i-oblast|moskva|moskovskaya-oblast).*')) THEN 'MSK_UP'
        ELSE 'OTHER_LP'
    END as region,
    case       
        when landingpagepath = '/' and medium like '%organic%' then 'Brand_organic'
        when landingpagepath = '/' then 'Brand_other'
        when (medium like '%cpc%' and (campaign  like '%search_second-building-discovery%' or campaign like '%second-building-smart-kms%')) then 'CPC_UP_Trash'
        WHEN campaign LIKE '%second-building%' then 'CPC_UP'
        WHEN campaign LIKE '%new-build%' then 'CPC_NB'
        else 'OTHER_SOURCE' 
    end AS type,
    concat(source, '/' , medium) as source_medium,
    count(distinct dimension1) as uniqs,
    count(distinct tr.dimension4) as sessions
FROM pv LEFT JOIN tr using(dimension4)
GROUP BY week, region, type, source_medium
HAVING week IS NOT NULL and week = {week}
ORDER BY week
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)


traf = traf.dropna(0)

traf_check = traf.values.tolist()
# traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
# wk.update("N1", traf_check)
wk.append_rows(traf_check, table_range="N2")

# Кол-во расхлопов и юзеры, сделавшие расхлоп без директа и с лендингом УП

In [ ]:
re, columns = client.execute('''
WITH 
tr as (
    SELECT dimension1, dimension4
    FROM ga.UA_TRAFIC_BIG 
    WHERE date >='2023-01-02' AND landingpagepath LIKE '%nedvizhimost%' AND source <> '(direct)' AND dimension1 IS NOT NULL
    GROUP BY dimension1, dimension4
),
     
ev AS (
    SELECT
        dateHourMinute, 
        SUM(totalEvents) as totalEvents, 
        dimension4
    FROM ga.RAW_EVENTS
    WHERE match(eventlabel, '(ClCardSellPhoneClickTop|ClSerpSellSecondVasPhoneClick|ClCardSellPhoneClickBottom|ClCardSellSecondVasPhoneClickAll|ClSerpCardSellNewDevPaidPhoneClick)')
        AND eventlabel NOT LIKE '%ClCardSellPhoneClickAll%' AND date >= '2023-01-02' 
    GROUP BY dateHourMinute, dimension4
)

SELECT
    toISOWeek(dateHourMinute) as isoweek, 
    SUM(totalEvents) AS totalEvents, 
    COUNT(DISTINCT dimension1) AS dimension1
FROM tr 
INNER JOIN ev ON tr.dimension4 = ev.dimension4
GROUP BY isoweek
ORDER BY isoweek
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()

sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("U1", traf_check)

# Расхлопы всех источников и по рекламе (по регионам)

In [ ]:
re, columns = client.execute('''
SELECT
    week,
    IFNULL(SUM(CASE WHEN city = 'MSK' AND cpc_type = 'УП реклама' THEN uniqueEvents END), 0) as msk_cpc,
    IFNULL(SUM(CASE WHEN city = 'MSK'  THEN uniqueEvents END), 0) as msk_all,
    IFNULL(SUM(CASE WHEN city = 'SPB' AND cpc_type = 'УП реклама' THEN uniqueEvents END), 0) as spb_cpc,
    IFNULL(SUM(CASE WHEN city = 'SPB'  THEN uniqueEvents END), 0) as spb_all,
    IFNULL(SUM(CASE WHEN city = '0' AND cpc_type = 'УП реклама' THEN uniqueEvents END), 0) as reg_cpc,
    IFNULL(SUM(CASE WHEN city = '0'  THEN uniqueEvents END), 0) as reg_all
FROM messnerav.CL_MAIN_REPORT_PHONE_CLICK
WHERE year = 2023
GROUP BY week
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("Y1", traf_check)

# Переходы в ИБ и пользователи

In [ ]:
re, columns = client.execute('''
WITH 
tr as (
    SELECT date, dimension4, dimension1, landingpagepath
    FROM ga.UA_TRAFIC_BIG 
    WHERE date >='2023-01-02' AND landingpagepath LIKE '%nedvizhimost%' AND dimension1 IS NOT NULL
    GROUP BY  date, dimension4, dimension1, landingpagepath
),
     
ev AS (
    SELECT eventlabel, dimension4, pagepath, SUM(totalEvents) as totalEvents
    FROM ga.RAW_EVENTS
    WHERE match(eventlabel, '(ClCardSellMortgageClick|ClSerpSellMortgageClick|^claim_completed)') AND date >= '2023-01-02'
    GROUP BY eventlabel, dimension4, pagepath
)

SELECT
    toISOWeek(date) as isoweek,
        CASE
        WHEN pagepath LIKE '%mosk%' THEN 'MSK'
        WHEN pagepath LIKE '%sankt-%' OR pagepath LIKE '%lening%' THEN 'SPB'
        ELSE 'REG'
    END as city_pp,
    IFNULL(SUM(CASE WHEN (eventlabel = 'ClCardSellMortgageClick' OR eventlabel = 'ClSerpSellMortgageClick') AND landingpagepath NOT LIKE '%ohio8.vchecks.me%' THEN totalEvents END), 0) as mortgage,
    COUNT(DISTINCT CASE WHEN (eventlabel = 'ClCardSellMortgageClick' OR eventlabel = 'ClSerpSellMortgageClick') THEN dimension1 END) AS dimension1,
    CASE
        WHEN landingpagepath LIKE '%mosk%' THEN 'MSK'
        WHEN landingpagepath LIKE '%sankt-%' OR landingpagepath LIKE '%lening%' THEN 'SPB'
        ELSE 'REG'
    END as city_lp,
    IFNULL(SUM(CASE WHEN eventlabel LIKE '%claim_completed%' THEN totalEvents END), 0) as claim_comp
FROM tr inner JOIN ev ON tr.dimension4=ev.dimension4
GROUP BY isoweek, city_lp, city_pp
ORDER BY isoweek
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf['mortgage'] = traf['mortgage'].astype(int)
traf['dimension1'] = traf['dimension1'].astype(int)
traf['claim_comp'] = traf['claim_comp'].astype(int)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()

sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("AG1", traf_check)

# Регистрации Все и с лендингом УП по ролям FL|P_AGENT

In [ ]:
re, columns = client.execute('''
WITH 
tr AS(
    SELECT dimension4, landingpagepath
    FROM  ga.UA_TRAFIC_BIG WHERE date >='2023-01-02' AND landingpagepath NOT LIKE '%ohio8.vchecks.me%' AND dimension1 IS NOT NULL
    GROUP BY  dimension4, landingpagepath
),
ev AS(
    SELECT date, eventlabel, dimension4, SUM(totalEvents) as totalEvents
    FROM ga.RAW_EVENTS
    WHERE match(eventlabel, '(NewRegSuccess)') AND match(eventCategory, '(FL|P_AGENT)') AND pagepath NOT LIKE '%link-registration%'
        AND date >= '2023-01-02'
    GROUP BY eventlabel, dimension4, date
)
SELECT
    toISOWeek(date) as isoweek,
    IFNULL(SUM(totalEvents), 0) as regis_all,
    IFNULL(SUM(CASE WHEN landingpagepath LIKE '%nedvizhimost%' THEN totalEvents END), 0) as regis_up
FROM tr inner JOIN ev ON tr.dimension4 = ev.dimension4
GROUP BY isoweek
ORDER BY isoweek''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("AN1", traf_check)

# Регистрации из ПГ (бэк)

In [ ]:
re, columns = client.execute('''
select
    toISOWeek(date) as isoweek,
    count(distinct user_code) as regs
from pg.PG_REGS
where role IN ('CLIENT', 'PROFESSIONAL') and date >= '2023-01-02'
    --and report_type= 'Зарегистрированный пользователь (установлен пароль)' НЕ нужно - исх.табла только по этим юзерам
GROUP BY isoweek
order by isoweek
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("AR1", traf_check)

# MAU для планерки

In [ ]:
re, columns = client.execute('''
SELECT
    month(date) as month,
    count(distinct dimension1) as uniqs
FROM ga.UA_TRAFIC_BIG
where landingpagepath like '%nedvizhimost%' AND medium <> '(none)' AND toISOYear(date) = 2023
group by month
order by month
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("AU1", traf_check)

# Расхлопы по VAS

In [ ]:
re, columns = client.execute('''
SELECT
    toISOWeek(date) as isoweek, 
    CASE
        WHEN pagepath LIKE '%mosk%' THEN 'MSK'
        WHEN pagepath LIKE '%sankt-%' OR pagepath LIKE '%lening%' THEN 'SPB'
        ELSE 'REG'
    END as city, 
    SUM(totalEvents) as tot, 
    SUM(uniqueEvents) as un
FROM ga.RAW_EVENTS 
WHERE date >= '2023-01-02' AND match(eventlabel, 'ClCardSellSecondVasPhoneClickAll|ClSerpSellSecondVasPhoneClick')
GROUP BY isoweek, city
ORDER BY isoweek, city
''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("AX1", traf_check)

# VAS по дням

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
             'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('m2-main-cd9ed0b4e222.json', SCOPES)
gc = gspread.authorize(credentials)

sh = gc.open('VAS - Партнёры')
wk = sh.worksheet('Ответы на форму (1)')
list_of_dicts = wk.get_all_records()
source = pd.DataFrame(list_of_dicts)

In [ ]:
df = source[['Отметка времени','Название агентства','Регион','Сумма договора','Дата старта','Выберите тип размещение на площадке']].copy()
df = df.rename(columns = {
 'Отметка времени':'timestamp_mark',
 'Название агентства':'agency', 
 'Регион':'region',
 'Сумма договора': 'deal_amount',
 'Дата старта': 'start_date',
 'Выберите тип размещение на площадке': 'placement_type',
}).fillna(0)
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')
df['deal_amount'] = df['deal_amount'].astype(int)
df['timestamp_mark'] = pd.to_datetime(df['timestamp_mark'])
df = df[df['placement_type'].isin(['Первичное размещение', 'Пролонгация текущего договора'])]
df = df.reset_index(drop=True)
df = df[df['timestamp_mark']!='']
df['deal_amount'] = df['deal_amount'] / 30
df['days'] = 30

df = df[['agency','region','start_date','deal_amount','days']].copy()

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')

agencies = df['agency'].unique()

for a in agencies:

    single_agency = df.query("agency == @a").sort_values('start_date')

    for index, row in single_agency.iterrows():
        _row = df.loc[[index]] # пересчитываем из оригинального df на случай если уже изменили это значение на предыдущих итерациях цикла
        start = _row['start_date'].iloc[0]
        in_a_deal = start + dt.timedelta(days = 30)
        overlaps = single_agency.query(
            "`start_date` < @in_a_deal and `start_date` > @start")

        if (len(overlaps)):

            for i, ov in overlaps.iterrows():

                print('Setting row {} to {}'.format(i, in_a_deal))
                df.at[i,'start_date'] = in_a_deal

In [ ]:
df['days'] = df['days'].astype(int)
cols = list(df.columns)+['date_spend']
res = []
for i in df.itertuples():    
    for date_diff in range(i.days):
        standart_row = list(i[1:])
        date_spend = i.start_date + dt.timedelta(days=date_diff)
        standart_row.append(date_spend)
        res.append(standart_row)
        
common = pd.DataFrame(res, columns = cols)

In [ ]:
common = common.drop(df.columns[[2, 4]], axis=1).copy()
common['date_spend'] = common['date_spend'].astype(str)

In [ ]:
common['deal_amount'] = common['deal_amount'].astype(float)
common = common[common['date_spend'] >= '2022-06-06']
common['agency'] = common['agency'].astype(str)
common['region'] = common['region'].astype(str)
common['date_spend'] = common['date_spend'].astype(str)

In [ ]:
traf_check =  [common.columns.values.tolist()] + common.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("BC1", traf_check)

In [ ]:
df.sort_values('start_date')

# Трафик по девайсам

In [ ]:
re, columns = client.execute('''
select
    toISOWeek(dateHourMinute) as week,
    deviceCategory,
    count(distinct dimension1) as users
from ga.UA_TRAFIC_BIG
where landingpagepath LIKE '%nedviz%' AND toISOYear(date) = 2023 AND toISOWeek(date) >= 1
GROUP by week, deviceCategory
ORDER by week, deviceCategory

''', with_column_types = True)


col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("BJ1", traf_check)

# Регистрация ЧМАКОВ (WEB) (дозапись)

In [ ]:
re, columns = client.execute('''
WITH
week(today()) - 1 as week1,

A AS (
    WITH
    page_view AS (
        SELECT dimension4, pagepath, MIN(dateHourMinute) AS page_fst_date
        FROM ga.PAGE_VIEWS
        WHERE match(pagepath, '(/nedvizhimost/)') AND dateHourMinute >= '2023-01-02'
        GROUP BY dimension4, pagepath
    ),
    sessions AS (
        SELECT  dimension1, dimension4
        FROM ga.UA_TRAFIC_BIG
        WHERE date >= '2023-01-02'
        GROUP BY dimension1, dimension4
    ),
    user AS (
        SELECT dimension1, dimension2
        FROM ga.USERS_DT
        WHERE date >= '2023-01-02'
        GROUP BY dimension1, dimension2
    ),
    pg_reg AS (
        SELECT user_code, registration_msk_ts as date
        FROM pg.PG_REGS
        WHERE role LIKE 'PROFESSIONAL' AND date >= '2023-01-02'
    )

    SELECT user_code
    FROM page_view
    LEFT JOIN sessions ON page_view.dimension4=sessions.dimension4
    LEFT JOIN user ON user.dimension1 = sessions.dimension1
    LEFT JOIN pg_reg ON  dimension2 = user_code
    WHERE page_fst_date < date AND toISOWeek(date) = week1 AND toISOYear(date) = 2023
    GROUP BY user_code
),

B AS (
    WITH
    page_view AS (
        SELECT dimension4, pagepath, MIN(dateHourMinute) AS page_fst_date
        FROM ga.PAGE_VIEWS
        WHERE match(pagepath, '(/rieltoram/placement)') AND dateHourMinute >= '2023-01-02'
        GROUP BY dimension4, pagepath
    ),
    sessions AS (
        SELECT  dimension1, dimension4
        FROM ga.UA_TRAFIC_BIG
        WHERE date >= '2023-01-02'
        GROUP BY dimension1, dimension4
    ),
    user AS (
        SELECT dimension1, dimension2
        FROM ga.USERS_DT
        WHERE date >= '2023-01-02'
        GROUP BY dimension1, dimension2
    ),
    pg_reg AS (
        SELECT user_code,  registration_msk_ts as date
        FROM pg.PG_REGS
        WHERE role LIKE 'PROFESSIONAL' AND date >= '2023-01-02'
    )

    SELECT user_code
    FROM page_view
    LEFT JOIN sessions ON page_view.dimension4=sessions.dimension4
    LEFT JOIN user ON user.dimension1 = sessions.dimension1
    LEFT JOIN pg_reg ON  dimension2 = user_code
    WHERE page_fst_date < date AND toISOWeek(date) = week1 AND toISOYear(date) = 2023
    GROUP BY user_code
)

SELECT
    week1 as week,
    count(distinct A.user_code) as nedv,
    count(distinct B.user_code) as riel,
    (SELECT COUNT(distinct user_code) as delta FROM A JOIN B USING(user_code)) as delta
FROM A
FULL JOIN B USING(user_code)
''', with_column_types = True)

col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = traf.values.tolist()
# traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
# wk.update("BN1", traf_check)
wk.append_rows(traf_check, table_range="BN2")

# Звонки по Новостройкам в УП (даш в отчете)

In [ ]:
re, columns = client.execute('''
with 
SESSIONS AS (
    select date, dimension1, dimension4
    from ga.UA_TRAFIC_BIG
    where landingpagepath LIKE '%nedvizhimost%' AND toISOYear(date) >= 2023
    group by date, dimension1, dimension4
),
     
NB_CLOPS AS(
    select dimension4
    from ga.RAW_EVENTS
    where match(eventlabel, 'NbCardPhoneNumberClick[0-9]|NbSnippetPhoneNumberClick|NbCardPhoneСallRequestSendClick|NbCardPhoneNumberClick')
        and toISOYear(date) >= 2023
    group by dimension4
),

CALL_GAINS AS (
    select
        session_id,
        SUM(calls) AS calls,
        SUM(matched_calls) AS matched_calls
    from mart.CALLS_FULL_ATRIBS
    where toISOYear(first_call_dt) >= 2023
    group by session_id
)

SELECT
    toISOWeek(date) AS week,
    COUNT(DISTINCT dimension1) AS user_id,
    SUM(CALL_GAINS.calls) AS calls,
    SUM(CALL_GAINS.matched_calls) AS matched_calls 
FROM (
    SELECT
        date,
        dimension1,
        CALL_GAINS.calls,
        CALL_GAINS.matched_calls
    FROM SESSIONS
    JOIN NB_CLOPS ON SESSIONS.dimension4 = NB_CLOPS.dimension4
    JOIN CALL_GAINS ON session_id = NB_CLOPS.dimension4
    ) 
WHERE CALL_GAINS.calls > 0
GROUP BY week
''', with_column_types = True)

col = []
for i in columns:
    col.append(i[0])
traf = pd.DataFrame(re,columns=col)

traf_check = [traf.columns.values.tolist()] + traf.values.tolist()
sh = gc.open("Classified.Reports 2023")
wk = sh.worksheet('source')
wk.update("BT1", traf_check)